<a href="https://colab.research.google.com/github/Metamad/example-repo/blob/main/Implementation_compl%C3%A8te_okV2nok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mémo‑titre pour le notebook**
Π_dlhash × ICICLE GPU —
Prouver “x · G” sans l’emuler : diviser le coût SNARK par 10 000


---


Problème. Les SNARKs travaillent dans Fₚ ; or la crypto réelle vit dans Z_q. Vérifier une seule multiplication scalaire elliptique (X = x·G) dans le circuit coûte ≈ 1,7 M contraintes.


---


Objectif. Implémenter le protocole Π_dlhash (Beyond the Circuit) : déplacer l’arithmétique elliptique hors‑circuit, ne laisser à Circom que
z = k + c·x + Poseidon2 → ≈ 300 contraintes.


---


Insight à démontrer. Poseidon2 calculé sur GPU (ICICLE) + Schnorr off‑circuit = preuve Groth16 légère, vérifiable on‑chain, sans gadgets non‑natifs.

Ce qu’on veut prouver.

Le circuit réduit vraiment le R1CS d’un facteur ≥ 10 000.

Le hash GPU abaisse le temps de preuve à < 50 ms.

Le schéma reste sûr (knowledge soundness) et portable à des use‑cases signature, bridge, ZK‑KYC.

(Tout le notebook déroule la chaîne complète : setup Colab GPU → ICICLE → Rust Π_dlhash → Circom → Groth16 → benchmarks.)

Cette cellule exécute nvidia‑smi, c’est‑à‑dire l’outil NVIDIA qui affiche en une ligne l’état du GPU (modèle, pilotes, version CUDA, température, mémoire disponible). Elle sert à vérifier que Colab a bien alloué un GPU compatible et que notre backend CUDA d’ICICLE pourra s’y connecter avant de lancer les calculs Poseidon sur GPU.

In [ ]:
!nvidia-smi

Mon Apr 21 08:37:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Cette cellule installe le socle d’outils système (git, gcc, make, cmake, pkg‑config, libclang, curl) indispensable pour cloner les dépôts, compiler les crates Rust/C++ d’ICICLE et exécuter les téléchargements — bref, elle prépare l’environnement Linux pour que tout le reste du notebook puisse se construire sans erreur.

In [ ]:
!apt-get update && apt-get install -y cmake git build-essential pkg-config libclang-dev curl


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois

Cette cellule installe et rend accessible la toolchain Rust dans l’environnement Colab.
Sans elle : cargo build échouerait, et toute la partie « compilation + exécution du hash Poseidon2 sur GPU » serait impossible.

In [ ]:
!curl https://sh.rustup.rs -sSf | sh -s -- -y
import os
os.environ["PATH"] += ":/root/.cargo/bin"


info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-04-03, rust version 1.86.0 (05f9846f8 2025-03-31)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
 21.2 MiB /  21.2 MiB (100 %)   2.7 MiB/s in  7s
info: installing component 'rust-std'
 27.1 MiB /  27.1 MiB (100 %)  10.2 MiB/s in  3s
info: installing component 'rustc'
 72.8 MiB /  72.8 MiB (100 %)   9.0 MiB/s in  8s
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu installed - rustc 1.86.0 (05f9846f8 2025-03-31)


Cette cellule installe le moteur GPU d’ICICLE en une minute au lieu de compiler tout le projet.
Ensuite, ton code Rust (zk_dlhash) pourra charger, via runtime::load_backend(…), la librairie /content/icicle/lib/backend/libposeidon2_bn254_cuda.so (ou équivalent) et exécuter le hash Poseidon2 directement sur le GPU T4 de Colab.

C’est donc la brique « accélération GPU » indispensable : sans cette archive, on resterait en mode CPU, et tout l’intérêt de Π_dlhash (hash ultra‑rapide hors‑circuit) serait perdu.

In [ ]:
%%bash
# 🧹 Nettoyage
rm -rf /content/icicle /content/icicle_3_7_0*

# 📥 Téléchargement de l'archive
curl -L -o icicle_3_7_0.tar.gz https://github.com/ingonyama-zk/icicle/releases/download/v3.7.0/icicle_3_7_0-ubuntu22-cuda122.tar.gz

# 📂 Extraction directe dans /content (pas de mv)
tar -xf icicle_3_7_0.tar.gz -C /content

# 🔍 Vérification
ls /content/icicle


lib


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  236M  100  236M    0     0  32.3M      0  0:00:07  0:00:07 --:--:-- 37.3M


Pour pouvoir exécuter Poseidon2 sur GPU et profiter des kernels multi‑threads d’ICICLE dans notre pipeline Π_dlhash, nous devons disposer :

du workspace Rust complet d’ICICLE (toutes les crates source) pour compiler notre binaire zk_dlhash, et

du backend CUDA pré‑compilé (lib/backend/*.so) qui contient les kernels GPU pour BN254.

La séquence ci‑dessous nettoie l’environnement Colab et garantit ces deux pré‑requis :

Clone récursif : embarque les modules icicle-core, icicle-bn254, icicle-hash, etc.

Sauvegarde / restauration des .so : évite de télécharger de nouveau les backends CUDA/Bare‑metal que nous avons déjà extraits plus tôt.

Chemin unique /content/icicle : simplifie les commandes suivantes (cargo build, cargo run).

En résumé, cette cellule prépare un workspace Rust prêt à compiler et conserve le moteur GPU indispensable à l’accélération Poseidon ; elle pose donc la fondation matérielle et logicielle sur laquelle repose toute l’implémentation de Π_dlhash dans le reste du notebook.

In [ ]:
%%bash

# Sauvegarde temporaire des .so (si présents)
mv /content/icicle/lib /content/icicle_lib_backup 2>/dev/null || true

# Supprime l'ancien dossier partiel
rm -rf /content/icicle

# Clone complet avec tous les crates Rust
git clone --recursive https://github.com/ingonyama-zk/icicle.git /content/icicle

# Restaure les .so (GPU backend)
mv /content/icicle_lib_backup /content/icicle/lib 2>/dev/null || true


Cloning into '/content/icicle'...


En résumé, cette cellule produit les binaires optimisés nécessaires à l’exécution GPU de Π_dlhash, et la durée de 18‑20 minutes vient de la compilation LLVM lourde d’un gros projet crypto + CUDA sans cache dans l’environnement Colab.

In [ ]:
%%bash
cd /content/icicle/wrappers/rust

# Compilation avec les bonnes features désactivées
cargo build --release --features=no_ecntt,no_g2


    Updating crates.io index
     Locking 127 packages to latest compatible versions
      Adding criterion v0.3.6 (available: v0.5.1)
      Adding lambdaworks-math v0.6.0 (available: v0.12.0)
      Adding rand v0.8.5 (available: v0.9.1)
      Adding risc0-core v0.21.0 (available: v2.0.0)
      Adding risc0-zkp v0.21.0 (available: v2.0.0)
  Downloaded once_cell v1.21.3
  Downloaded hex v0.4.3
  Downloaded rand_core v0.6.4
  Downloaded shlex v1.3.0
  Downloaded getrandom v0.2.15
  Downloaded cc v1.2.19
  Downloaded crossbeam-utils v0.8.21
  Downloaded rayon-core v1.12.1
  Downloaded rand v0.8.5
  Downloaded zerocopy v0.8.24
  Downloaded crossbeam-epoch v0.9.18
  Downloaded crossbeam-deque v0.8.6
  Downloaded rayon v1.10.0
  Downloaded cmake v0.1.54
  Downloaded rand_chacha v0.3.1
  Downloaded ppv-lite86 v0.2.21
  Downloaded either v1.15.0
  Downloaded cfg-if v1.0.0
  Downloaded libc v0.2.172
   Compiling shlex v1.3.0
   Compiling libc v0.2.172
   Compiling cc v1.2.19
   Compiling crossb

cette cellule pose la “boîte” Rust dans laquelle tu vas implémenter le protocole Π_dlhash, tout en l’intégrant au workspace ICICLE pour bénéficier automatiquement des dépendances (icicle‑core, icicle‑bn254, etc.) et de la compilation GPU.

In [ ]:
import os

os.system("cargo new --bin /content/icicle/wrappers/rust/zk_dlhash")


0

sans ce fichier correctement rempli, le compilateur ne saurait pas où trouver les briques ICICLE et refuserait de construire notre programme. Cette cellule prépare donc le terrain pour que la compilation se passe sans erreur

In [ ]:
with open("/content/icicle/wrappers/rust/zk_dlhash/Cargo.toml", "w") as f:
    f.write("""
[package]
name = "zk_dlhash"
version = "0.1.0"
edition = "2021"

[dependencies]
icicle-runtime = { path = "../icicle-runtime" }
icicle-core = { path = "../icicle-core" }
icicle-bn254 = { path = "../icicle-curves/icicle-bn254" }
hex = "0.4"
""".strip())


In [ ]:
with open("/content/icicle/wrappers/rust/zk_dlhash/src/main.rs", "w") as f:
    f.write("""
use icicle_bn254::curve::{CurveCfg, G1Projective, ScalarCfg};
use icicle_core::{curve::Curve, msm, msm::MSMConfig, traits::GenerateRandom};
use icicle_runtime::{device::Device, memory::HostSlice, runtime};

fn main() {
    // 🔧 Backend CUDA
    runtime::load_backend("/tmp/bn254_backend/icicle/lib/backend").unwrap();
    let device = Device::new("CPU", 0);
    icicle_runtime::set_device(&device).unwrap();

    // 🎯 MSM simple (x⋅G hors-circuit, preuve dans circuit plus tard)
    let size = 1024;
    let points = CurveCfg::generate_random_affine_points(size);
    let scalars = ScalarCfg::generate_random(size);

    let mut result = vec![G1Projective::zero(); 1];
    msm::msm(
        HostSlice::from_slice(&scalars),
        HostSlice::from_slice(&points),
        &MSMConfig::default(),
        HostSlice::from_mut_slice(&mut result),
    ).unwrap();

    println!("✅ MSM[{}] result = {:?}", size, result[0]);
}
""".strip())


In [ ]:
%cd /content/icicle/wrappers/rust/zk_dlhash
!ICICLE_BACKEND_INSTALL_DIR=/tmp/bn254_backend/icicle/lib/backend cargo run --release


/content/icicle/wrappers/rust/zk_dlhash
   Compiling icicle-bn254 v3.7.0 (/content/icicle/wrappers/rust/icicle-curves/icicle-bn254)
   Compiling zk_dlhash v0.1.0 (/content/icicle/wrappers/rust/zk_dlhash)
    Finished `release` profile [optimized] target(s) in 2m 44s
     Running `/content/icicle/wrappers/rust/target/release/zk_dlhash`
✅ MSM[1024] result = Projective { x: 0x05b6f989807c1fdfede0fc7500a7ed7ffa35272e286193a0e7aac6eaf258d910, y: 0x0297e8b83db21142306ae5af7299ef913e193183fcd775e9a615fa3cc593c788, z: 0x0a299a7972bc67d43bf85df4eb2f61ba19dd9762921d3e5e2bf12b00696b05ce }


In [ ]:
with open("/content/icicle/wrappers/rust/zk_dlhash/Cargo.toml", "w") as f:
    f.write("""
[package]
name = "zk_dlhash"
version = "0.1.0"
edition = "2021"

[dependencies]
icicle-runtime = { path = "../icicle-runtime" }
icicle-core = { path = "../icicle-core" }
icicle-bn254 = { path = "../icicle-curves/icicle-bn254" }
icicle-babybear = { path = "../icicle-fields/icicle-babybear" }
hex = "0.4"
rand = "0.8"
num-bigint = "0.4"
""".strip())



In [ ]:
# 📄 Écriture du circuit Circom pour Π_dlhash
with open("/content/circuit_dlhash.circom", "w") as f:
    f.write("""
pragma circom 2.1.6;

include "circomlib/poseidon.circom";

template DlHash() {
    // Inputs privés
    signal input x;
    signal input k;

    // Inputs publics
    signal input z;
    signal input hash;

    // Interne : Poseidon(x, k)
    component hasher = Poseidon(2);
    hasher.inputs[0] <== x;
    hasher.inputs[1] <== k;

    // Challenge Fiat-Shamir
    signal c;
    c <== hasher.out;

    // Vérification dans le circuit
    z === k + c * x;
    hash === c;
}

component main = DlHash();
""".strip())


Cette cellule installe l’outil “circom” (le compilateur de circuits ZK) et, pour y parvenir, installe Node + npm.
Sans ce bloc, on ne pourrait pas transformer le fichier .circom en .wasm/.r1cs, donc impossible de calculer le witness ou de prouver quoi que ce soit.

In [ ]:
%%bash
# 📦 Installe circom via npm (Node.js)
apt-get update -y
apt-get install -y nodejs npm
# ⛔️ on n’installe plus Circom via npm :
# npm install -g circom   # <-- à supprimer/commenter


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
The following additional

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
npm WARN deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you want a good and tested way to coalesce async requests by a key value, which is much more comprehensive and powerful.
npm WARN deprecated glob@7.1.6: Glob versions prior to v9 are no longer supported
npm WARN deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported
npm WARN deprecated rimraf@2.7.1: Rimraf versions prior to v4 are no longer supported
npm WARN deprecated circom@0.5.46: Package no longer supported. Contact Support at https://www.npmjs.com/support for more info.


Cette cellule remplace la vieille Circom installée via npm par une version moderne, rapide et native, compilée directement depuis les sources officielles.

In [ ]:
# Supprimer l'ancien Circom
!npm uninstall -g circom

# Installer les dépendances
!sudo apt update && sudo apt install -y build-essential

# Cloner la bonne version
!git clone https://github.com/iden3/circom.git /content/circom-src
%cd /content/circom-src
!git checkout v2.1.4

# Compiler depuis les sources
!cargo build --release

# Lier le binaire manuellement
!ln -sf /content/circom-src/target/release/circom /usr/local/bin/circom
!circom --version




removed 133 packages, and audited 1 package in 880ms

found 0 vulnerabilities
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
39 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'mai

In [ ]:
!circom /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash.circom --r1cs --wasm --sym -I /content/circomlib/circomlib


error: Found argument '-I' which wasn't expected, or isn't valid in this context

USAGE:
    circom --r1cs --sym --wasm

For more information try --help


In [ ]:
!git clone https://github.com/iden3/circomlib.git /content/circomlib


Cloning into '/content/circomlib'...
remote: Enumerating objects: 4769, done.
remote: Counting objects: 100% (1176/1176), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 4769 (delta 1023), reused 949 (delta 948), pack-reused 3593 (from 1)
Receiving objects: 100% (4769/4769), 9.26 MiB | 15.14 MiB/s, done.
Resolving deltas: 100% (2995/2995), done.


In [ ]:
!find /content/circomlib -name poseidon.circom


/content/circomlib/circuits/poseidon.circom


In [ ]:
# ✅ Écriture du fichier main.rs corrigé
with open("/content/icicle/wrappers/rust/zk_dlhash/src/main.rs", "w") as f:
    f.write("""
use icicle_core::{hash::HashConfig, poseidon::Poseidon, traits::FieldImpl};
use icicle_runtime::{device::Device, memory::HostSlice, runtime};
use icicle_bn254::curve::ScalarField;
use rand::RngCore;
use std::fs::File;
use std::io::Write;

// 🔄 Génère un scalaire aléatoire dans le champ natif
fn random_scalar<R: RngCore>(rng: &mut R) -> ScalarField {
    let mut bytes = [0u8; 32];
    rng.fill_bytes(&mut bytes);
    ScalarField::from_bytes_le(&bytes)
}

// 🔄 Convertit un scalaire en chaîne décimale pour input.json
fn to_dec_string(scalar: &ScalarField) -> String {
    let bytes = scalar.to_bytes_le();
    let num = num_bigint::BigUint::from_bytes_le(&bytes);
    num.to_string()
}

fn main() {
    // 🚀 Initialisation du backend GPU (ou CPU fallback)
    runtime::load_backend("/tmp/bn254_backend/icicle/lib/backend").unwrap();
    let device = Device::new("CPU", 0); // "CUDA" si dispo
    icicle_runtime::set_device(&device).unwrap();

    // 📏 Paramètres du hash
    let t = 3;
    let batch = 1;

    // 🔐 Secrets : x et k
    let mut rng = rand::thread_rng();
    let x = random_scalar(&mut rng);
    let k = random_scalar(&mut rng);

    // 💠 Hash Poseidon(x, k, 0)
    let inputs = vec![x, k, ScalarField::zero()];
    let mut outputs = vec![ScalarField::zero(); batch];
    let hasher = Poseidon::new::<ScalarField>(t, None).unwrap();
    hasher.hash(
        HostSlice::from_slice(&inputs),
        &HashConfig::default(),
        HostSlice::from_mut_slice(&mut outputs),
    ).unwrap();

    let c = outputs[0];
    let z = k + c * x;

    // ✅ Affichage
    println!("✅ Π_dlhash inputs:");
    println!("x  = {}", x);
    println!("k  = {}", k);
    println!("c  = Poseidon(x,k) = {}", c);
    println!("z  = k + c·x = {}", z);

    // 📄 Génération du fichier input.json pour Circom
    let json = format!(
        "{{\\n  \\"x\\": \\"{}\\",\\n  \\"k\\": \\"{}\\",\\n  \\"z\\": \\"{}\\",\\n  \\"hash\\": \\"{}\\"\\n}}",
        to_dec_string(&x),
        to_dec_string(&k),
        to_dec_string(&(k + c * x)), // sécurité redondante
        to_dec_string(&c)
    );

    let mut file = File::create("/content/input.json").unwrap();
    file.write_all(json.as_bytes()).unwrap();
}
""".strip())


TEST A SUPPRIMER SI PAS OK

In [ ]:
%%bash
set -e                                                         # stop on error

############### 1) main.rs (écrasement complet) ###############
cat > /content/icicle/wrappers/rust/zk_dlhash/src/main.rs <<'RS'
use icicle_core::{hash::HashConfig, poseidon::Poseidon, traits::FieldImpl};
use icicle_runtime::{device::Device, memory::HostSlice, runtime};
use icicle_bn254::curve::ScalarField;
use rand::RngCore;
use std::{fs::File, io::Write};

fn rand_scalar<R: RngCore>(rng: &mut R) -> ScalarField {
    let mut bytes = [0u8; 32];
    rng.fill_bytes(&mut bytes);
    ScalarField::from_bytes_le(&bytes)
}

fn to_dec(s: &ScalarField) -> String {
    use num_bigint::BigUint;
    BigUint::from_bytes_le(&s.to_bytes_le()).to_string()
}

fn main() {
    // ⚠️  ne charge QUE le backend BN254/CPU
    runtime::load_backend("/content/icicle/lib/backend/bn254/cpu").unwrap();
    icicle_runtime::set_device(&Device::new("CPU", 0)).unwrap();

    // secrets aléatoires
    let mut rng = rand::thread_rng();
    let x = rand_scalar(&mut rng);
    let k = rand_scalar(&mut rng);

    // Poseidon‑2 (t = 3)
    let hasher = Poseidon::new::<ScalarField>(3, None).unwrap();
    let mut out = vec![ScalarField::zero()];
    hasher
        .hash(
            HostSlice::from_slice(&[x, k, ScalarField::zero()]),      // (x,k,0)
            &HashConfig::default(),
            HostSlice::from_mut_slice(&mut out),
        )
        .unwrap();
    let c = out[0];
    let z = k + c * x;

    println!("✅ Π_dlhash inputs:");
    println!("x  = {}", x);
    println!("k  = {}", k);
    println!("c  = Poseidon2(x,k) = {}", c);
    println!("z  = k + c·x = {}", z);

    // écrit input.json au format décimal pour Circom/snarkjs
    let json = format!(
        "{{\"x\":\"{}\",\"k\":\"{}\",\"z\":\"{}\",\"hash\":\"{}\"}}",
        to_dec(&x),
        to_dec(&k),
        to_dec(&z),
        to_dec(&c)
    );
    File::create("/content/input.json")
        .unwrap()
        .write_all(json.as_bytes())
        .unwrap();
}
RS
echo "✅  main.rs mis à jour"

############### 2) compilation #################################
cd /content/icicle/wrappers/rust
cargo build --release -p zk_dlhash
echo "✅  binaire compilé"

############### 3) exécution isolée ############################
export ICICLE_BACKEND_INSTALL_DIR=/content/icicle/lib/backend/bn254/cpu
export LD_LIBRARY_PATH=$ICICLE_BACKEND_INSTALL_DIR:${LD_LIBRARY_PATH:-}

./target/release/zk_dlhash         # ⇒ doit afficher x, k, c, z et écrire /content/input.json


✅  main.rs mis à jour
✅  binaire compilé
✅ Π_dlhash inputs:
x  = 0x353c5eb44178831f17bb7965b90cba52e8d24c99a5b5ace800c6a3ab983e9736
k  = 0xc8d0b70e60d5868333baf58b0bea4886ca17f59a3b317e7214e504921ee17c16
c  = Poseidon2(x,k) = 0x1ecb623000227b7ca86cb9674db8362d12dba055c416b664021a2637c8f596a1
z  = k + c·x = 0xbf955005b685a974e68a6fca38c9c3d36aaed1c69513f3be8d7befadbd96f136


   Compiling zk_dlhash v0.1.0 (/content/icicle/wrappers/rust/zk_dlhash)
    Finished `release` profile [optimized] target(s) in 0.50s


In [ ]:
%%bash
set -euxo pipefail

###############################################################################
# 0) chemins ------------------------------------------------------------------
###############################################################################
CIRCUIT_DIR=/content/icicle/wrappers/rust/zk_dlhash
RAW_BASE=https://raw.githubusercontent.com/iden3/circomlib2/master/circuits/hash

###############################################################################
# 1) poseidon2.circom (+ dépendances)  ----------------------------------------
###############################################################################
for file in poseidon2.circom poseidon.circom constants.circom
do
  dst="$CIRCUIT_DIR/$file"
  [[ -f "$dst" ]] && continue
  echo "→ download $file"
  curl -sSfL "$RAW_BASE/$file" -o "$dst"
done

###############################################################################
# 2) circuit DLHash2 (Poseidon‑2, t = 2) --------------------------------------
###############################################################################
cat > "$CIRCUIT_DIR/circuit_dlhash.circom" <<'CIR'
pragma circom 2.1.4;

include "poseidon2.circom";

template DLHash2() {
    signal input x;
    signal input k;
    signal input z;
    signal input hash;

    component h = Poseidon2(2);           // arité 2  (x , k)
    h.inputs[0] <== x;
    h.inputs[1] <== k;

    signal c <== h.out;

    hash === c;                           // vérifie le hash fourni
    z    === k + c * x;                   // contrainte principale
}
component main = DLHash2();
CIR

echo "→ compile circuit"
circom "$CIRCUIT_DIR/circuit_dlhash.circom" --wasm --r1cs --sym \
      -l "$CIRCUIT_DIR" -o "$CIRCUIT_DIR"

###############################################################################
# 3) src/main.rs (Rust : Poseidon‑2, t = 2) -----------------------------------
###############################################################################
cat > "$CIRCUIT_DIR/src/main.rs" <<'RS'
use icicle_core::{
    hash::HashConfig,
    poseidon::{Poseidon, PoseidonVariant},
    traits::FieldImpl,
};
use icicle_runtime::{device::Device, memory::HostSlice, runtime};
use icicle_bn254::curve::ScalarField;
use rand::RngCore;
use std::{fs::File, io::Write};

fn rand_scalar<R: RngCore>(rng: &mut R) -> ScalarField {
    let mut b = [0u8; 32];
    rng.fill_bytes(&mut b);
    ScalarField::from_bytes_le(&b)
}
fn to_dec(s: &ScalarField) -> String {
    use num_bigint::BigUint;
    BigUint::from_bytes_le(&s.to_bytes_le()).to_string()
}

fn main() {
    runtime::load_backend("/content/icicle/lib/backend").unwrap();
    icicle_runtime::set_device(&Device::new("CPU", 0)).unwrap();

    let mut rng = rand::thread_rng();
    let x = rand_scalar(&mut rng);
    let k = rand_scalar(&mut rng);

    // ── Poseidon‑2, t = 2 ────────────────────────────────────────────────
    let hasher = Poseidon::new::<ScalarField>(
        2,
        Some(PoseidonVariant::Poseidon2)
    ).unwrap();
    let mut out = vec![ScalarField::zero()];
    hasher.hash(
        HostSlice::from_slice(&[x, k]),
        &HashConfig::default(),
        HostSlice::from_mut_slice(&mut out),
    ).unwrap();

    let c = out[0];
    let z = k + c * x;

    println!("✅ Π_dlhash inputs:");
    println!("x  = {}", x);
    println!("k  = {}", k);
    println!("c  = Poseidon2(x,k) = {}", c);
    println!("z  = k + c·x = {}", z);

    let json = format!(
        "{{\n  \"x\":\"{}\",\n  \"k\":\"{}\",\n  \"z\":\"{}\",\n  \"hash\":\"{}\"\n}}",
        to_dec(&x), to_dec(&k), to_dec(&z), to_dec(&c)
    );
    File::create("/content/input.json").unwrap().write_all(json.as_bytes()).unwrap();
}
RS

###############################################################################
# 4) compile & exécute le binaire Rust  ---------------------------------------
###############################################################################
cd /content/icicle/wrappers/rust
cargo build --release -p zk_dlhash
./target/release/zk_dlhash

###############################################################################
# 5) witness (snarkjs) --------------------------------------------------------
###############################################################################
npx -y snarkjs wtns calculate \
      "$CIRCUIT_DIR/circuit_dlhash_js/circuit_dlhash.wasm" \
      /content/input.json \
      /content/witness.wtns

echo -e "\n🎉  witness.wtns généré → /content/witness.wtns"


→ compile circuit


+ CIRCUIT_DIR=/content/icicle/wrappers/rust/zk_dlhash
+ RAW_BASE=https://raw.githubusercontent.com/iden3/circomlib2/master/circuits/hash
+ for file in poseidon2.circom poseidon.circom constants.circom
+ dst=/content/icicle/wrappers/rust/zk_dlhash/poseidon2.circom
+ [[ -f /content/icicle/wrappers/rust/zk_dlhash/poseidon2.circom ]]
+ continue
+ for file in poseidon2.circom poseidon.circom constants.circom
+ dst=/content/icicle/wrappers/rust/zk_dlhash/poseidon.circom
+ [[ -f /content/icicle/wrappers/rust/zk_dlhash/poseidon.circom ]]
+ continue
+ for file in poseidon2.circom poseidon.circom constants.circom
+ dst=/content/icicle/wrappers/rust/zk_dlhash/constants.circom
+ [[ -f /content/icicle/wrappers/rust/zk_dlhash/constants.circom ]]
+ continue
+ cat
+ echo '→ compile circuit'
+ circom /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash.circom --wasm --r1cs --sym -l /content/icicle/wrappers/rust/zk_dlhash -o /content/icicle/wrappers/rust/zk_dlhash
error[P1000]: UnrecognizedToken { tok

CalledProcessError: Command 'b'set -euxo pipefail\n\n###############################################################################\n# 0) chemins ------------------------------------------------------------------\n###############################################################################\nCIRCUIT_DIR=/content/icicle/wrappers/rust/zk_dlhash\nRAW_BASE=https://raw.githubusercontent.com/iden3/circomlib2/master/circuits/hash\n\n###############################################################################\n# 1) poseidon2.circom (+ d\xc3\xa9pendances)  ----------------------------------------\n###############################################################################\nfor file in poseidon2.circom poseidon.circom constants.circom\ndo\n  dst="$CIRCUIT_DIR/$file"\n  [[ -f "$dst" ]] && continue\n  echo "\xe2\x86\x92 download $file"\n  curl -sSfL "$RAW_BASE/$file" -o "$dst"\ndone\n\n###############################################################################\n# 2) circuit DLHash2 (

In [ ]:
# ✏️  Remplace les 4 chaînes hexa par celles imprimées par ton exécutable
x_hex = "0xc7333cc4df2b057d59a7c60c40eeb6768029011ab98cd42715ad89017e546dad"
k_hex = "0x7f7068e2bcadc813b964463c633c107d3c2a9e2ca28abea3fea9881868fe0e75"
c_hex = "0x21e6014d83b0a25d0a3f6a9b278cb3b587f5c47b5a4351b8c367fbc91d326e49"
z_hex = "0x6754b93239ca473877bfd931b0db490664adc0660e48635f50fdf8610f7e65c4"

def h2d(h):                            # hex → décimal (string)
    return str(int(h, 16))

input_data = {
    "x":    h2d(x_hex),
    "k":    h2d(k_hex),
    "hash": h2d(c_hex),
    "z":    h2d(z_hex),
}

import json, pathlib, pprint
pathlib.Path("/content/input.json").write_text(json.dumps(input_data))
print("✅ input.json écrit :")
pprint.pprint(input_data)


✅ input.json écrit :
{'hash': '15332707819677632690374936580028774414718243577176525926191010743847903981129',
 'k': '57642342535768223780216453680985306267588769627235824479101299510471582158453',
 'x': '90100785480806402120058638878251707680941481277908922514707686989859803917741',
 'z': '46737916734675324950277617707000744727489932644434892186739954708153165047236'}


FLOW OK

In [ ]:
!cd /content/icicle/wrappers/rust/zk_dlhash && \
ICICLE_BACKEND_INSTALL_DIR=/tmp/bn254_backend/icicle/lib/backend cargo run --release


    Finished `release` profile [optimized] target(s) in 0.08s
     Running `/content/icicle/wrappers/rust/target/release/zk_dlhash`
✅ Π_dlhash inputs:
x  = 0x1aed8e447d49a19b2b5ca93d6d44967a083e5903ecbcec78fdd110021b923db0
k  = 0x34e3ea46285dcacda773531ce3e647601dcd215d1e53641b914673ed5b0a9cff
c  = Poseidon2(x,k) = 0x1f3b9221a6e64c7914f08b0e9a1f7e71cf3cdb9571d63f3e68b6ecc2dbb27e33
z  = k + c·x = 0x1d717b6ed9b4f76e912f4aaef5ca885e7dbca2f4e42e96bb54874b38a14147ca


In [ ]:
# Supprimer l'ancien fichier circuit (par précaution)
!rm -f /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash.circom

# ✅ Nouveau circuit avec Poseidon(3) + vérification explicite
circuit_code = [
    "pragma circom 2.1.4;",
    "",
    'include "./poseidon.circom";',  # ✅ Inclure Poseidon localement
    "",
    "template DLHash() {",
    "    signal input x;",
    "    signal input k;",
    "    signal input z;",
    "    signal input hash;",
    "",
    "    signal c;",
    "    component h = Poseidon(3);",  # Arity 3
    "    h.inputs[0] <== x;",
    "    h.inputs[1] <== k;",
    "    h.inputs[2] <== 0;",          # Padding explicite
    "    c <== h.out;",
    "",
    "    signal expected;",
    "    expected <== k + c * x;",
    "",
    "    hash === h.out;",             # ✅ Hash vérifié
    "    expected === z;",             # ✅ Contrainte critique",
    "}",
    "",
    "component main = DLHash();"
]

# Écriture du fichier
with open("/content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash.circom", "w", encoding="utf-8", newline="\n") as f:
    for line in circuit_code:
        f.write(line + "\n")

print("✅ Circuit Circom mis à jour avec Poseidon(3) et contraintes explicites.")


✅ Circuit Circom mis à jour avec Poseidon(3) et contraintes explicites.


In [ ]:
!circom /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash.circom \
  --r1cs --wasm --sym \
  -l /content/circomlib/circuits

error[P1000]: Include not found: ./poseidon.circom

previous errors were found


Créer un input.json valide

In [ ]:
import re, json

rust_output = """
✅ Π_dlhash inputs:
x  = 0x28378a9e
k  = 0x4423ad0d
c  = Poseidon(x,k) = 0x6e7e4b6c
z  = k + c·x = 0x69ea89b7
"""

def extract_hex(name, line):
    m = re.search(r"0x[0-9a-f]+", line)
    return m.group() if m else None

lines = rust_output.strip().splitlines()
parsed = {
    "x": extract_hex("x", lines[1]),
    "k": extract_hex("k", lines[2]),
    "hash": extract_hex("c", lines[3]),
    "z": extract_hex("z", lines[4]),
}

with open("/content/input.json", "w") as f:
    json.dump(parsed, f)

print("✅ input.json :")
print(json.dumps(parsed, indent=2))


✅ input.json :
{
  "x": "0x28378a9e",
  "k": "0x4423ad0d",
  "hash": "0x6e7e4b6c",
  "z": "0x69ea89b7"
}


In [ ]:
# Derniers résultats depuis Rust
x_hex = "0x94f8b68dbb222902043955c038c0394a76d215770512103a720966c46447373d"
k_hex = "0x01380549a30a10934bdf991257944e80cace9df085d94e39345c71b31f87ed99"
c_hex = "0x03b5e35fb282bc74f300aa47573d86d467e591d1d543f960e34bb2681e4e2da9"
z_hex = "0x036af52e841a29c0ed564971cea1e94eb197965e89a748938f7f5231d29674eb"

def hex_to_dec(hex_str):
    return str(int(hex_str, 16))

input_data = {
    "x": hex_to_dec(x_hex),
    "k": hex_to_dec(k_hex),
    "z": hex_to_dec(z_hex),
    "hash": hex_to_dec(c_hex)
}

import json
with open("/content/input.json", "w") as f:
    json.dump(input_data, f)

print("✅ input.json mis à jour au format décimal")


✅ input.json mis à jour au format décimal


Génère le witness avec le bon chemin

In [ ]:
!circom /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash.circom \
  --r1cs --wasm --sym \
  -l /content/circomlib/circuits \
  -o /content/icicle/wrappers/rust/zk_dlhash



error[P1000]: Include not found: ./poseidon.circom

previous errors were found


In [ ]:
!find /content -name "circuit_dlhash.wasm"



/content/circom-src/circuit_dlhash_js/circuit_dlhash.wasm
/content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/circuit_dlhash.wasm


In [ ]:
!npx snarkjs wtns calculate \
  /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/circuit_dlhash.wasm \
  /content/input.json \
  /content/witness.wtns





⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼Need to install the following packages:
snarkjs@0.7.5
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏[ERROR] snarkJS: [Error: ENOENT: no such file or directory, open '/content/input.json'] {
  errno: -2,
  code: 'ENOENT',
  syscall: 'open',
  path: '/content/input.json'
}
⠙npm notice
npm notice New major version of npm available! 10.8.2 -> 11.3.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.3.0
npm notice To update run: npm install -g npm@11.3.0
npm notice
⠙

In [ ]:
!node /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/generate_witness.js \
  /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/circuit_dlhash.wasm \
  /content/input.json \
  /content/witness.wtns




(node:23220) UnhandledPromiseRejectionWarning: Error: Error: Assert Failed.
Error in template DLHash_70 line: 21

    at /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/witness_calculator.js:161:27
    at Array.forEach (<anonymous>)
    at WitnessCalculator._doCalculateWitness (/content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/witness_calculator.js:136:14)
    at WitnessCalculator.calculateWTNSBin (/content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/witness_calculator.js:212:20)
    at /content/icicle/wrappers/rust/zk_dlhash/circuit_dlhash_js/generate_witness.js:15:38
(node:23220) UnhandledPromiseRejectionWarning: Unhandled promise rejection. This error originated either by throwing inside of an async function without a catch block, or by rejecting a promise which was not handled with .catch(). To terminate the node process on unhandled promise rejection, use the CLI flag `--unhandled-rejections=strict` (see https://nodejs.org/api/cli.html#cli_unhandled_rejectio